# Student Loan Risk with Deep Learning

In [39]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [41]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
df = pd.read_csv(file_path)

# Review the DataFrame
df

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,0
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,0
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,1
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,0
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,1
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,1
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,0


In [43]:
# Review the data types associated with the columns
df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [45]:
# Check the credit_ranking value counts
df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [48]:
# Define the target set y using the credit_ranking column
y = df['credit_ranking']
# Display a sample of y
y

0       0
1       0
2       0
3       1
4       0
       ..
1594    0
1595    1
1596    1
1597    0
1598    1
Name: credit_ranking, Length: 1599, dtype: int64

In [50]:
# Define features set X by selecting all columns but credit_ranking
X = df.drop(['credit_ranking'], axis = 1)

# Review the features DataFrame
X

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


### Step 3: Split the features and target sets into training and testing datasets.


In [53]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [56]:
# Create a StandardScaler instance
std_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaled_train = std_scaler.fit_transform(X_train)

# Fit the scaler to the features training dataset


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [60]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
input_nodes

11

In [72]:
# Define the number of hidden nodes for the first hidden layer
hidden_1_nodes = 6
# Define the number of hidden nodes for the second hidden layer
hidden_2_nodes = 6
# Define the number of neurons in the output layer
output_nodes = 1

In [74]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# input layer
nn_model.add(tf.keras.layers.InputLayer(shape=(input_nodes,)))

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_1_nodes, activation="relu"))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_2_nodes, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=output_nodes, activation="sigmoid"))

In [76]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 6)              │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 121 (484.00 B)

 Trainable params: 121 (484.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [78]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [80]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train, y_train, epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 505us/step - accuracy: 0.4674 - loss: 4.6201
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 434us/step - accuracy: 0.4749 - loss: 1.7860
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 378us/step - accuracy: 0.3777 - loss: 1.0608
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 290us/step - accuracy: 0.3819 - loss: 0.9617
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 337us/step - accuracy: 0.4025 - loss: 0.8599
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 313us/step - accuracy: 0.3832 - loss: 0.8161
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 298us/step - accuracy: 0.4440 - loss: 0.7568
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 364us/step - accuracy: 0.4358 - loss: 0.7323
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 289us/step - accuracy: 0.4763 - loss: 0.7098
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 286us/step - accuracy: 0.4988 - loss: 0.6927
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 327us/step - accuracy: 0.5714 - loss: 0.6742
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 318us/step

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [82]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data

model_loss, model_accuracy = nn_model.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Display the model loss and accuracy results


13/13 - 0s - 4ms/step - accuracy: 0.6225 - loss: 0.6404
Loss: 0.6404244303703308, Accuracy: 0.6225000023841858


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [84]:
# Set the model's file path
file_path = Path("/Users/georgearistides/Desktop/M18_Starter_Code/student_loans.keras")

# Export your model to a keras file
nn_model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [86]:
# Set the model's file path
file_path = Path("/Users/georgearistides/Desktop/M18_Starter_Code/student_loans.keras")

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [96]:
# Make predictions with the test data
predictions = nn_imported.predict(X_test,verbose=2)

# Display a sample of the predictions
predictions

13/13 - 0s - 1ms/step


array([[0.52915657],
       [0.35643992],
       [0.59600466],
       [0.5199474 ],
       [0.65200984],
       [0.6215016 ],
       [0.60567087],
       [0.16183357],
       [0.560731  ],
       [0.6298886 ],
       [0.70640415],
       [0.5274912 ],
       [0.40291202],
       [0.6186646 ],
       [0.45418823],
       [0.44368407],
       [0.7494691 ],
       [0.640997  ],
       [0.64828146],
       [0.5860764 ],
       [0.5625093 ],
       [0.48247957],
       [0.6000226 ],
       [0.5988024 ],
       [0.30768433],
       [0.7871915 ],
       [0.5975121 ],
       [0.43848944],
       [0.48800078],
       [0.00424102],
       [0.5787778 ],
       [0.610061  ],
       [0.26248893],
       [0.6344429 ],
       [0.33581412],
       [0.46533915],
       [0.6171789 ],
       [0.6308127 ],
       [0.60942304],
       [0.6142094 ],
       [0.7921036 ],
       [0.06832134],
       [0.4485545 ],
       [0.84271514],
       [0.11178449],
       [0.61706305],
       [0.47706664],
       [0.624

In [102]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_rounded = [round(prediction[0],0) for prediction in predictions]

predictions_df = pd.DataFrame(predictions_rounded, columns=['Predictions'])
predictions_df

,Predictions
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,1.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [108]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df))

              precision    recall  f1-score   support

           0       0.62      0.51      0.56       188
           1       0.62      0.72      0.67       212

    accuracy                           0.62       400
   macro avg       0.62      0.62      0.61       400
weighted avg       0.62      0.62      0.62       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

The data provided gives a number of metrics related to payment history, performance in school, financial aid, and the specific field being studied. These are all important indicators of tendency and ability to pay off debts before/during study, as well as for forecasting the range of income a student may make after graduating based on performance and choice of field, which could be very useful. If a student chooses a field that is highly competetitive but also highly lucrative, performance in school could be a large indicator of if that individual could pay off their debts quickly; the current market for computer science graduates is a prime example, where there is potential to start with some of the highest base salaries, but lack of positions and a market that is oversaturated with new graduates means in most cases only a few top performers will fall in this category. The caveat is that some metrics may inadvertently correlate credibility with socioeconomic/ethinc status based on things like location, so that would need to be addressed or acknowledged.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

This would be context based filtering. Simply put, this takes into context a number of features for each individual which are taken into context with the target of credit worthy for a loan. For example metrics like gpa or major may not obviously be correlated with credit, however for reasons mentioned in the previous question they are important in context, like many other features.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

The biggest challenge would be bias - as aforementioned - towards or against individuals with certain socioeconomic stati or ethnic backgrounds due to correlations between features and these statistics. A model may inadvertently correlate these pieces of information with loan payability vicariously through given features and as a result would discriminate against certain groups.

The other big challenge I can see would be doing a better job of this than a human would - the only real benefit I see with such a use case would be predicting on large amounts of data, however due to the aforementioned challenge I believe having a human review cases is necessary. The inherent bias based on statistics in these cases is too great, and I think there is a level of judgement and subjectivity to these binary decisions that should not be done by machine.
